<a href="https://colab.research.google.com/github/PECGPT/PECGPT/blob/main/entrevistador_v2_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai > /dev/null 2>&1
from google.colab import drive, userdata, output, files
from openai import OpenAI
import sys
import os
import string
from IPython.display import display, HTML


testando = False
testandofinal = False
usarGPT4 = True

#interface gráfica do Colab:
html_code = '''
<style>
#container {
  display: flex;
  flex-direction: column;
  height: 90vh;
}
#output_area {
  flex: 1;
  overflow-y: auto;
  background: #f8f8f8;
  border: 1px solid #ccc;
  padding: 8px;
  min-height: 300px;
}
#input_area {
  margin-top: 10px;
}
</style>
<div id="container">
  <div id="output_area" rows="30"></div>
  <div id="input_area">
    <textarea id="user_input" rows="4" cols="150" maxlength="800" placeholder=" "></textarea><br>
    <button onclick="sendInput()">Enviar</button>
  </div>
</div>
<script>
  function sendInput() {
    const input = document.getElementById('user_input').value;
    google.colab.kernel.invokeFunction('notebook.handle_input', [input], {});
    document.getElementById('user_input').value = ''; // Clear textarea after sending
  }
    user_input.addEventListener('keydown', function(event) {
    if (event.key === 'Enter' && !event.shiftKey) { // Check if Enter is pressed without the Shift key
      event.preventDefault(); // Prevent the default action to avoid a new line
      sendInput(); // Call the function to submit the text
    }
  });

</script>
'''

display(HTML(html_code))

class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

class OutputCapture:
    def __init__(self, element_id):
        self.element_id = element_id

    def write(self, msg):
        script = f'''
        let box = document.getElementById('{self.element_id}');
        box.innerHTML += `{msg}<br/>`; // Append new messages as HTML
        '''
        output.eval_js(script)

    def flush(self):
        pass

"""
CASO ESTEJA PUXANDO DO GOOGLE DRIVE (atualmente biblioteca no Github):
    drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive')
"""



OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)



#busca criterios4.py do Github
with SuppressOutput():
      !git clone https://github.com/PECGPT/PECGPT.git
sys.path.append('/content/PECGPT')
from criterios4 import *

sys.stdout = OutputCapture('output_area')

#variáveis globais
i = 0  # contador de mensagens
max_mensagens = 20
prompt_tokens_35 = 0
completion_tokens_35 = 0
prompt_tokens_4 = 0
completion_tokens_4 = 0
criterios = criterios_ativos

#históricos de mensagem:
message_history = [] #message history completo
message_history_cand = [] #apenas as mensagens com o candidato

message_history_final = [] #usado para as avaliações finais.
### implementar isto aqui.

#tirar: ler de criterios.py
#inst_inicial = """Neste estágio inicial da entrevista, não aborde a escolha de curso diretamente. Tente conhecer melhor o perfil do candidato, sua familiaridade e interesse no Brasil, e seus hobbies e interesses pessoais.\n Exemplos de primeiras perguntas possíveis: 'Você já esteve no Brasil?' 'Que cidades e regiões do Brasil você tem mais vontade de conhecer?'; 'Por que você escolheu estudar no exterior?' 'Você considerou estudar em algum país além do Brasil?'; 'Quais são suas matérias favoritas na escola?'; 'Qual é seu livro favorito e por que você gosta dele?'; 'Quem você listaria como suas inspirações pessoais?'\n Não fique muito tempo em um só assunto: às vezes faça perguntas relacionadas à resposta anterior, e às vezes introduza um assunto novo. Tenha cuidado para não fazer perguntas redundantes, sobre coisas que o candidato já disse em mensagens anteriores, mesmo que falando de outro tema. Aborde um assunto por vez."""


def chat(inp, papel, modelo="gpt-3.5-turbo", encerra = False):

    global prompt_tokens_35, completion_tokens_35, prompt_tokens_4, completion_tokens_4, testando, usarGPT4
    if not usarGPT4: modelo = "gpt-3.5-turbo"
    if papel == "candidato":
        inp = "CAND: " + inp
        role = "user"
        message_history_cand.append({"role": role, "content": f"{inp}"})
    elif papel == "supervisor":
        inp = "SUPX: " + inp
        role = "user"
    elif papel == "system":
        role = "system"
        message_history_cand.append({"role": role, "content": f"{inp}"})
    message_history.append({"role": role, "content": f"{inp}"})


    if encerra:
      message_history_final.append({"role": role, "content": f"{inp}"})
      msg_hist = message_history_final
    else:
      msg_hist = message_history

    completion = client.chat.completions.create(
        model=modelo,
        messages = msg_hist
    )

    reply_content = completion.choices[0].message.content

    if papel == "candidato":
      message_history_cand.append({"role": "assistant", "content": f"{reply_content}"})
    if encerra:
      message_history_final.append({"role": role, "content": f"{reply_content}"})
    if papel != "system":
      message_history.append({"role": "assistant", "content": f"{reply_content}"})



    if modelo == "gpt-3.5-turbo":
      prompt_tokens_35 += completion.usage.prompt_tokens
      completion_tokens_35 += completion.usage.completion_tokens
    else:
      prompt_tokens_4 += completion.usage.prompt_tokens
      completion_tokens_4 += completion.usage.completion_tokens

    return reply_content


def checa_resposta_int(s):
    try:
        int(s)
        return True
    except ValueError:
        return False






class Supervisor:
  global message_history, message_history_final, message_history_cand, inst_sistema, inst_inicial
  inst = ""

  def __init__(self, criterios, criterios_adicionais, inst, nome):
    self.criterios = criterios
    self.criterios_adicionais = criterios_adicionais
    self.num_criterios = len(criterios)
    self.num_criterios_adicionais = len(criterios_adicionais)
    self.num_mensagens_iniciais = 4
    self.freq_revisao_criterio = 4
    self.avaliacoes = [-1 for _ in range(self.num_criterios)]
    self.certezas = [0 for _ in range(self.num_criterios)]
    self.avaliacoes_ad = [-1 for _ in range(self.num_criterios_adicionais)]
    self.criterio_atual = self.define_criterio(0, self.certezas)
    self.final = False
    self.instrucoes = inst
    self.relatorio = ""
    self.relatorio_apendice = ""
    self.nome = nome
    chat(self.instrucoes["sistema"], "system")


  def chama_supervisor(self, i, user_input): #RETORNA TRUE SE GERAR MENSAGEM.
    global max_mensagens, testando
    if testando: print("chama supervisor chamado com i = " + str(i))
    if i == 0:
      self.instrucao_inicial()
      return True
    if i == self.num_mensagens_iniciais:
      self.criterio_atual = self.define_criterio(i, self.certezas)
      self.informa_criterio(self.criterio_atual, user_input)
      return True
    if i > self.num_mensagens_iniciais and i % self.freq_revisao_criterio == self.num_mensagens_iniciais % self.freq_revisao_criterio and i < max_mensagens:
#      self.certezas[self.criterio_atual], self.avaliacoes[self.criterio_atual] = self.avalia_certeza(self.criterio_atual, user_input), self.avalia_criterio(self.criterio_atual, user_input)
      self.certezas[self.criterio_atual] = self.avalia_certeza(self.criterio_atual, user_input)
      if testando: print(self.certezas, self.avaliacoes)
      self.criterio_atual = self.define_criterio(i, self.certezas)
      if self.criterio_atual == -1:
        if testando: print("encerra entrevista chamado por esgotamento dos critérios em chama_supervisor.")
        self.encerra_entrevista(user_input)
      else:
        self.informa_criterio(self.criterio_atual, user_input)
        return True #?
    if i == max_mensagens:
        if testando: print("encerra entrevista chamado por máx de mensagens em chama_supervisor.")
        self.encerra_entrevista(user_input)
        return True
    return False

  def define_criterio(self, i, certezas):
     if i < self.num_mensagens_iniciais:
      return 0
     menor = min(certezas)
     if menor == 3:
       return -1 # -1 indica final da entrevista, já que todos os critérios estão preenchidos com certeza máxima.
     else:
      if testando: print("O próximo critério é: ", certezas.index(menor))
      return certezas.index(menor)

  def informa_criterio(self, crit, user_input):
    mensagem = "Ignore minha última instrução, pois passamos para outra fase da entrevista. Nesta fase da entrevista a prioridade é: \n\n" + self.criterios[crit]["prompt_entrevista"]
    mensagem += "\n\nAgora você vai retomar a conversa com o candidato. Suas mensagens devem seguir o objetivo acima. Lembre: você deve responder perguntas ou solicitações de informação APENAS do supervisor, não do candidato. \
    Se o candidato fizer perguntas ou solicitar informações que não sejam relacionadas à entrevista ou solicitar informação de qualquer outro tipo, diga gentilmente que não pode responder e recomende que visite o site do PEC-G, converse com o operador no Posto diplomático brasileiro mais próximo ou outra fonte relevante. A última \
    mensagem do candidato foi: CAND: " + user_input + " Responda esta mensagem com uma pergunta para o candidato, conforme as orientações acima. Não inclua nada além da mensagem para o candidato."
#    print(mensagem)
    resposta = chat(mensagem, "supervisor")
    if testando: print("critério em avaliação atualizado para " + str(crit) + " pela função informa_criterio")
    print(resposta)
    #criar mensagem de reforço com system?

  def instrucao_inicial(self):
    mensagem = self.instrucoes["inicial"] + "\nResponda esta mensagem com sua primeira mensagem para o candidato, iniciando a entrevista. Não inclua nada além da sua mensagem para o candidato."
    resposta = chat(mensagem, "supervisor")
    if testando: print("instrucao_inicial chamado, com mensagem inicial: " + mensagem)
    print(resposta)


  def avalia_certeza(self, crit, user_input, encerra=False):
    mensagem = ""
#    prompt_local = "responda apenas com um dígito: '3' se todos os itens foram abordados e o candidato forneceu informação detalhadas sobre todos os pontos, e/ou prestou informações tais que nenhuma outra pegunta sobre o tema seria pertinente no contexto da entrevista; '2' se todos os quase todos foram abordados, mas as informações fornecidas pelo candidato não são muito detalhadas; '1' se diversos itens importantes não foram abordados ainda, ou o candidato não voluntariou informação suficiente sobre parte significativa deles; e '0' se todos ou a maior parte dos itens para considerar nessa avaliação não foram abordados ou foram respondidos de forma vaga ou evasiva pelo candidato."

    if testando: print(f"avalia_certeza chamado com encerra = {encerra} e crit = {crit}")

    if not encerra:
      mensagem += "CAND: " + user_input + "\n"
      mensagem += "SUPX: a última mensagem do candidato está acima. Agora, vamos fazer uma pausa na conversa com o candidato para uma avaliação.\n"

#    mensagem += "SUPX: Revise agora toda sua conversa com o candidato e considere quanta informação foi fornecida pelo candidato, para avalia-lo no critério a seguir.\n"
    mensagem += "Descrição do critério: " + self.criterios[crit]["descrição"] + ".\n\n"
    mensagem += self.criterios[crit]["prompt_certeza"] + "\n"
    mensagem += self.instrucoes["certeza_1"]
    if testando: print("Consulta de critérios: " + mensagem)

    if not encerra:
      resp = chat(mensagem, "supervisor")
      if testando: print("avaliação de certeza chamada. Mensagem: " + mensagem + "\n\nResposta: " + resp + "\n")
#      chat(mensagem, "supervisor")
      mensagem = "SUPX: " + self.instrucoes["certeza_2"]
      resposta = chat(mensagem, "supervisor")
      if testando: print("avaliação de certeza chamada. Segunda leva de mensagens: Mensagem: " + mensagem + "\n\nResposta: " + resposta + "\n")

      if not checa_resposta_int(resposta):
        mensagem = "Responda apenas com um dígito numérico, conforme instruído. Não inclua mais nada na sua mensagem, apenas um caractere numérico."
        resposta = chat(mensagem, "supervisor") #ver se ok com o 3.5
      cert = int(resposta) #TODO: revisar para o caso de precisar insistir; fazer um loopzinho com checa_resposta_int

    if encerra:
      resposta = chat(mensagem, "supervisor", modelo="gpt-4o-2024-05-13", encerra=True)
      if testando: print(resposta)
      mensagem = "SUPX: " + self.instrucoes["certeza_2"]
      resposta = chat(mensagem, "supervisor", modelo="gpt-4o-2024-05-13", encerra=True)

      if not checa_resposta_int(resposta):
        mensagem = "Responda apenas com um dígito numérico, correspondente à categoria."
        resposta = chat(mensagem, "supervisor") #ver se ok com o 3.5

      cert = int(resposta) #TODO: revisar para o caso de precisar insistir; fazer um loopzinho com checa_resposta_int

    return cert

  def avalia_criterio(self, crit, user_input, enc=False):
    global testandofinal
    mensagem = ""
    if testando: print("avalia_criterio chamado com encerra = True e crit = " + str(crit))
    if not enc:
      mensagem += "\"CAND: " + user_input + "\"\n"
      mensagem += "SUPX: a última mensagem do candidato está acima. Agora, vamos fazer uma pausa na conversa com o candidato para uma avaliação.\n"
#      if testando: print("avalia_criterio chamado com encerra = False")

    mensagem += "Revise agora, cuidadosamente, todo o histórico da sua conversa com o candidato. " + self.criterios[crit]["prompt_avaliação"] + "\n Classifique o candidato em uma dessas categorias: "  + self.criterios[crit]["categorias"] + " Antes de classificar o candidato, explique seu raciocínio sobre cada um dos pontos levantados para essa avaliação, atendo-se ao que foi conversado entre você e o candidato. Esteja atento para nuances: há informações incorretas ou vagas nas falas do candidato? O candidato mostrou-se reticente ao abordar algum ponto? O candidato pode estar sendo otimista demais sobre sua situação financeira? Ao final, junte todas as suas considerações e escolha uma das categorias."
    resposta = chat(mensagem, "supervisor", modelo="gpt-4o-2024-05-13", encerra=enc)
    if enc:
       self.relatorio_apendice += "\n\nANÁLISE SOBRE O CRITÉRIO " + self.criterios[crit]["nome"].upper() + ":\n\n " + resposta + "\n\n"
       if testandofinal:
              print("mensagem para avalia_criterios:", mensagem, "resposta: ", resposta)
    mensagem = "Agora, condense sua mensagem anterior informando apenas o dígito que identifica a categoria em que está classificando o candidato. Não inclua mais nada na sua mensagem, apenas um caractere numérico para indicar a categoria."
    resposta = chat(mensagem, "supervisor", encerra=enc)
    if testando: print("resposta foi: ", resposta)
    if not checa_resposta_int(resposta):
      mensagem = "Responda apenas com um dígito numérico, correspondente à categoria."
      resposta = chat(mensagem, "supervisor") #TODO: revisar para o caso de precisar insistir; fazer um loopzinho com checa_resposta_int

    result = int(resposta)
    return result



  def avalia_criterio_adicional(self, crit):
    global testandofinal
    if testando: print(f"avalia_criterio_adicional chamado com  crit = {crit}")

    mensagem = """SUPX: A entrevista acabou. Agora, vamos avaliar alguns critérios mais delicados, que exigirão uma leitura atenta de toda a conversa com o candidato até agora.
    Considere que o candidato pode não estar falando a verdade, pode ter um otimismo injustificado ou estar se baseando em informações equivocadas.
    É um trabalho de detetive! Preste muita atenção a pistas que o candidato pode ter dado sem querer. Este é um dos 'critérios especiais'\n\n"""

    mensagem += self.criterios_adicionais[crit]["prompt_avaliação"] + "\n Veja se há informação suficiente, nas mensagens do candidato, para classificá-lo em uma dessas categorias. Se houver, escolha uma categoria e justifique detalhadamente sua resposta."
    resposta = chat(mensagem, "supervisor", modelo="gpt-4o-2024-05-13", encerra=True)
    self.relatorio_apendice += "\n\nAVALIAÇÃO SOBRE O CRITÉRIO ADICIONAL " + self.criterios_adicionais[crit]["nome"].upper() + ":\n\n" + resposta + "\n\n"
    if testandofinal:
        print("\n\n\nmensagem para avalia_criterio_adicional:", mensagem, "\n\n\nresposta: ", resposta)
    mensagem = "Agora, condense sua mensagem anterior informando apenas o dígito que identifica a categoria em que está classificando o candidato. Não inclua mais nada na sua mensagem, apenas um caractere numérico para indicar a categoria. Se não houver informação suficiente para essa avaliação, responda -1"
    resposta = chat(mensagem, "supervisor", encerra=True)
    #if testando: print("Mensagem de consulta sobre critério adicional: ", mensagem)

    if not checa_resposta_int(resposta):
      mensagem = "Responda apenas com um dígito numérico, correspondente à categoria."
      resposta = chat(mensagem, "supervisor") #ver se ok com o 3.5
    result = int(resposta) #revisar para o caso de precisar insistir; fazer um loopzinho com checa_resposta_int
    return result


  def encerra_entrevista(self, user_input):
    global i, prompt_tokens_35, completion_tokens_35, prompt_tokens_4, completion_tokens_4, testando, testandofinal, message_history_cand, message_history_final #TODO: global i pq? Muita coisa global, fazer uma classe para msghistory

    if testando:
      print(f"encerra_entrevista chamado com self.final ={self.final}.")
    try:
      message_history_final = list(message_history_cand)
    except:
      print("falha no manuseio dos message histories!")

    if self.final:
      return True

    self.relatorio = "Candidato: " + self.nome + "\n\n\n"

    mensagem = "\n CAND: " + user_input + "\n"
    message_history_final.append({"role": "user", "content": f"{mensagem}"})
    message_history_cand.append({"role": "user", "content": f"{mensagem}"})
    mensagem += "SUPX: a última mensagem do candidato está acima. Já podemos encerrar a entrevista. Responda com uma mensagem para o candidato encerrando a entrevista de forma cordial e desejando boa sorte." #checar o fluxo no caso em que encerra_entrevista é chamado por critérios esgotados.
    resposta = chat(mensagem, "supervisor")
    print(resposta)
    print("\n\n\n####\n A entrevista acabou. Gerando o arquivo de análise (aguarde)")

    for i in range(self.num_criterios):
        if testando: print(f"entrou no loop com encerra = True e num_criterios = {self.num_criterios}. Está em {i}")
        self.certezas[i] = self.avalia_certeza(i, "", encerra=True)
        self.avaliacoes[i] = self.avalia_criterio(i, "", enc=True)
        if testandofinal: print(f"critério {self.criterios[i]['nome']} classificado na categoria {self.avaliacoes[i]} com certeza {self.certezas[i]}")
        self.relatorio += f"critério {self.criterios[i]['nome']} classificado na categoria {self.avaliacoes[i]} com certeza {self.certezas[i]} \n"

    for i in range (self.num_criterios_adicionais): #TODO: um toggle aqui se não houver critérios adicionais, por modularidade.
        self.avaliacoes_ad[i] = self.avalia_criterio_adicional(i)
        if testandofinal: print("critério adicional " + self.criterios_adicionais[i]["nome"] + " classificado na categoria " + str(self.avaliacoes_ad[i]))
        self.relatorio += "critério adicional" + self.criterios_adicionais[i]["nome"] + " classificado na categoria " + str(self.avaliacoes_ad[i]) + "\n"

    mensagem = self.instrucoes["resumo"]
    resposta = chat(mensagem, "supervisor", modelo="gpt-4o-2024-05-13")
#    print("\n\nPerfil do candidato:\n\n", resposta)
    self.relatorio += "PERFIL DO CANDIDATO: \n\n " + resposta + "\n\n\n"
    print("Avaliação nos critérios principais: ", self.certezas, self.avaliacoes + "\n\n")
    print("Avaliação nos critérios adicionais: ", self.avaliacoes_ad + "\n\n\n\n\n")
    if testando:
      print("message history com o candidato: ", message_history_cand)
      print("message history para avaliação final: ", message_history_final)

    self.relatorio += self.relatorio_apendice
    arquivo = self.nome +  ".txt"
    print(arquivo)
    with open(arquivo, "w") as file:
        file.write(self.relatorio)
        files.download(arquivo)
    arquivo = self.nome + " historico.txt"
    with open(arquivo, "w") as file:
        file.write(message_history_final)
        files.download(arquivo)
    print(f"\nArquivos {self.nome}.txt e {arquivo} gerados.")

#    if testandofinal:
    if True:
        print(f"Uso de tokens:\n prompts 3.5: {prompt_tokens_35}, completions 3.5: {completion_tokens_35}, prompt 4o: {prompt_tokens_4}, completion 4o: {completion_tokens_4}")
        custo = prompt_tokens_35 * 0.0000005 + completion_tokens_35 * 0.0000015 + prompt_tokens_4 * 0.000005 + completion_tokens_4 * 0.000015
        print("custo estimado: " + str(custo))
    self.final = True





#setup inicial. TODO: Alterar depois quando tiver como puxar os dados do candidato.
mensagem_inicial = "Olá! Informe seu nome para começar.\n" #TODO: revisar depois.
print(mensagem_inicial)
message_history.append({"role": "assistant", "content": f"{mensagem_inicial}"})
message_history_cand.append({"role": "assistant", "content": f"{mensagem_inicial}"})
nome = ""

supervisor = Supervisor(criterios, criterios_adicionais, inst, nome)

"""
for i in range(15):
    if not supervisor.final:
     user_input = input("> ")
    if i == 0:
       message_history.append({"role" : "user", "content": "EST: "f"{user_input}"})
    if not supervisor.chama_supervisor(i, user_input): # A chamada ao supervisor retorna True quando já resulta na mensagem seguinte ao candidato.
      print(chat(user_input, "candidato"))
    if supervisor.final: break
    print()
"""


def handle_input(user_input):
    global i, nome
    print("> " + user_input)
    if i == 0:
        message_history.append({"role": "user", "content": f"CAND: {user_input}"}) #TODO: duplicado? necessário?
        message_history_cand.append({"role": "user", "content": f"CAND: {user_input}"}) #TODO: duplicado? necessário?
        # Create translation table
        translator = str.maketrans('', '', string.punctuation)
        supervisor.nome = user_input.translate(translator)
 #       i = 19 #usado para pular para o final para fins de teste.
    if supervisor.final:
        return
    if not supervisor.chama_supervisor(i, user_input):
        response = chat(user_input, "candidato")
        print(response)
    if i >= max_mensagens:
        if supervisor.final == False:
            supervisor.encerra_entrevista(user_input)
        return
    i += 1

output.register_callback('notebook.handle_input', handle_input)



Mounted at /content/drive
